# Evaluation

## Preliminaries

### Imports

In [1]:
import sys
import os
sys.path.append('./src')

In [2]:
import pandas as pd

In [3]:
from programmable_chatbot.data.corpora import HOPE, IEMOCAP
from programmable_chatbot.chatbot_api import Chatbot

In [4]:
import numpy as np

In [5]:
import torch

In [6]:
import random

### Constants

In [7]:
DATA_PATH = '../resources/data/cache'

In [8]:
TOKENIZER = 'gpt2'
MODEL_PATH = '../experiments/PROGCHAT/gpt2_large_2023_01_06_12_19_33/model/best_checkpoint'

In [9]:
OUT_DF_COLUMNS = ['Split', 'Corpus', 'Conversation ID', 'Turn IDX', 'Speaker', 'Context', 'Last message', 'Response', 'Model']

In [30]:
GENERATE_KWARGS = {
    'top_p': 0.9, 'top_k': 0, 'temperature': 1.0, 'do_sample': True, 'max_new_tokens': 128
}

In [21]:
N_SAMPLES = 100

In [22]:
RANDOM_SEED = 2307

### Helper functions

In [23]:
def find_prompt(prompts, tgt):
    for p in prompts:
        if tgt in p:
            return p
    return -1

### Model and tokenizer

In [24]:
chatbot = Chatbot(
    MODEL_PATH,
    TOKENIZER
)

### Random seed

In [25]:
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

## Data

### Load data

In [31]:
hope = HOPE(
    '../resources/data/raw/HOPE_WSDM_2022',
    'test',
    chatbot.tokenizer,
    max_chunk_turns=8,
    max_context_turns=3,
    min_turns=3,
    sample=10
)
data_hope = hope.get_data_for_evaluation()

iemocap = IEMOCAP(
    '../resources/data/raw/IEMOCAP_full_release',
    'test',
    chatbot.tokenizer,
    max_chunk_turns=8,
    max_context_turns=3,
    min_turns=3,
    sample=10
)
data_iemocap = iemocap.get_data_for_evaluation()

### Randomly sample conversations

In [32]:
eval_data = {
    'HOPE': random.choice(data_hope['discriminator']['local']['dialogue_act']['infilling']),
    'IEMOCAP': random.choice(data_iemocap['discriminator']['local']['emotion']['infilling'])
}

## Evaluation data preparation

### Responses generation


In [33]:
out_data = []

for corpus, sample in eval_data.items():
    idx = random.choice(range(len(sample['utterances'])))
    task = sample['task_description']

    response = f'{find_prompt(sample["utterances"][idx]["prompts"], sample["utterances"][idx]["target"])}{sample["utterances"][idx]["text"]}'

    context = [
        f'{find_prompt(utterance["prompts"], utterance["target"])}{utterance["text"]}' for utterance in sample['utterances'][:idx]
    ]

    responses = []

    for prompt in sample["utterances"][idx]["prompts"]:
        responses.append((prompt, chatbot.generate(
            context,
            prompt=prompt,
            task_description=task,
            **GENERATE_KWARGS
        )))

    out_data.append((corpus, context, response, responses))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

In [34]:
out_data

[('HOPE',
  ["Therapist [dialogue act: information delivery]: Alright, so let's see if we could just talk about the moment of the presentation again. I want to try to help you develop some strategies to bring those anxiety symptoms down.\n",
   'Patient [dialogue act: acknowledgement]: Okay, right.\n',
   "Therapist [dialogue act: information delivery]: Let's actually go right before the moment so you're still in the, you're still sitting down, and you're getting ready to present. Let's just say your second present. Yeah, like we've talked about and you're thinking they're going to seeming as anxious. They're gonna think I don't belong here. These are the right. Okay. So these thoughts occur because you have an expectation rule one add to that you brought into that class, when you walked into that class, you have these expectations already. It's perfectly natural. It's not right or wrong. Everyone has these beliefs. And whenever these beliefs are and we'll talk about that another point

In [35]:
idx

29